# Notebook to generate psychometrics plot given sessions & conditions

In [4]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()


Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [5]:
import datajoint as dj
import pandas as pd
import numpy as np
import pylab as plt
import matplotlib.patches as mpatches

from matplotlib import cm
from u19_pipeline import utility
from inspect import getmembers, isfunction

### Datajoint configuration and Connection to DB

In [6]:

dj.conn()
acquisition = dj.create_virtual_module('acquisition', 'u19_acquisition')
pupillometry = dj.create_virtual_module('pupillometry', 'u19_pupillometry')

Connecting alvaros@datajoint00.pni.princeton.edu:3306


## Filter params

In [11]:
import pickle5 as pickle
#with open(path_to_protocol5, "rb") as fh:
#   data = pickle.load(fh)

file = '/Users/alvaros/Documents/MATLAB/BrainCogsProjects/Datajoint_proj/U19-pipeline_python/notebooks/jounhong_TH_276_20220406_g0_pupil.pkl'
#object = pd.read_pickle(file)
#object


objects = []
with (open(file, "rb")) as openfile:
    while True:
        try:
            object = pickle.load(openfile)
        except EOFError:
            break


pupillometry_data_array = object.to_numpy()


In [15]:
key = dict()
key['subject_fullname'] = 'jounhong_TH_276'
key['session_date'] = '2022-04-06'

new_key = (acquisition.Session & key).fetch1("KEY")
new_key['pupil_diameter'] = pupillometry_data_array

pupillometry.PupillometryData.insert1(new_key, allow_direct_insert=True)